In [1]:
import sys
import os
import pandas as pd

project_root = os.path.abspath(os.path.join(os.getcwd() , '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

trajectory_changes = f"{project_root}/data/coordinates/rdp_filtered_ger_vs_sp_coordinates.csv"
hand_position = f"{project_root}/data/coordinates/germany-vs-spain-cutted_hand_elbow_keypoints.csv"


In [2]:
trajectory_changes_df = pd.read_csv(trajectory_changes)
hand_position_df = pd.read_csv(hand_position)

In [3]:
hand_position_df_filtered = hand_position_df[hand_position_df['frame'].isin(trajectory_changes_df['frame'])]

hand_position_df_filtered['x'] = hand_position_df_filtered['x'].astype(int)
hand_position_df_filtered['y'] = hand_position_df_filtered['y'].astype(int)


/tmp/ipykernel_7368/1301859584.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hand_position_df_filtered['x'] = hand_position_df_filtered['x'].astype(int)
/tmp/ipykernel_7368/1301859584.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hand_position_df_filtered['y'] = hand_position_df_filtered['y'].astype(int)


In [4]:
hand_position_df_filtered

,frame,person_id,body_part,x,y,confidence
0,0,0,left_elbow,1731,489,0.973693
1,0,0,right_elbow,1513,426,0.982471
2,0,0,left_wrist,1736,517,0.961070
3,0,0,right_wrist,1453,449,0.968641
4,0,1,left_elbow,931,401,0.972225
...,...,...,...,...,...,...
3919,129,7,right_wrist,1404,370,0.603718
3920,129,8,left_elbow,399,322,0.511885
3921,129,8,right_elbow,311,330,0.861991
3922,129,8,left_wrist,0,0,0.394718


In [5]:
trajectory_changes_df

,frame,x,y
0,0,878,685
1,32,1130,760
2,91,1455,606
3,105,1435,738
4,129,1376,560


In [6]:
import pandas as pd
import numpy as np

# Set the tolerance for Euclidean distance
tolerance = 40.0

# Get common frame numbers
common_frames = set(trajectory_changes_df['frame']).intersection(set(hand_position_df_filtered['frame']))

# Collect results
results = []

for frame in common_frames:
    # Get reference point from trajectory_changes_df for this frame
    ref_row = trajectory_changes_df[trajectory_changes_df['frame'] == frame][['x', 'y']].iloc[0]
    ref_point = np.array([ref_row['x'], ref_row['y']])
    
    # Get all points from hand_position_df_filtered for this frame
    frame_points = hand_position_df_filtered[hand_position_df_filtered['frame'] == frame][['x', 'y']]
    
    # Check distance between each point and the reference point
    for _, row in frame_points.iterrows():
        point = np.array([row['x'], row['y']])
        distance = np.linalg.norm(point - ref_point)
        
        if distance <= tolerance:
            results.append({
                'frame': frame,
                'hand_x': row['x'],
                'hand_y': row['y'],
                'trajectory_x': ref_point[0],
                'trajectory_y': ref_point[1],
                'distance': distance
            })
            break  # Optional: stop at first match per frame

# Create a DataFrame with the matches
matching_df = pd.DataFrame(results)
print(matching_df)



   frame  hand_x  hand_y  trajectory_x  trajectory_y   distance
0    129    1409     553          1376           560  33.734256


In [7]:
import cv2

video_path = f"{project_root}/data/raw/germany-vs-spain-cutted.mp4"
cap = cv2.VideoCapture(video_path)

total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

frame_number = 92
cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
success, frame = cap.read()

if success:
    cv2.imwrite(f'{project_root}/data/detected_handspiel_pic/germany-vs-spain-handspiel.jpg', frame)  # Save as image
    print(f'Saved to: {project_root}/data/detected_pic/germany-vs-spain-handspiel.jpg')
else:
    print(f"Failed to retrieve frame {frame_number}")


Saved to: /home/denis/Projects/var-ai/data/detected_pic/germany-vs-spain-handspiel.jpg
